In [1]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
prov_codes = {'Alberta': 403,'Saskatchewan':639,'Saskatchewan/Alberta':403,'Quebec':514,'Ontario':249,'Ontario/Quebec':249,'Manitoba':204,'New Brunswick':506,'New Brunswick/Quebec':506,'Newfoundland and Labrador':709,'Northwest Territories':867,'Prince Edward Island':902,'Nova Scotia':902,'British Columbia':778}
Rooms = {'Bachelor units':0,'One bedroom units':1,'Two bedroom units':2,'Three bedroom units':3}
dataset = pd.read_csv('/https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/canada-rent/canada-rent.csv')

dataset = dataset[dataset['VALUE'].notna()]
dataset.rename(columns={'REF_DATE':'YEAR','VALUE':'MONTHLY RENT'},inplace=True)
dataset['PROVINCE'] = dataset['GEO'].map(lambda x: x.split(', ')[-1])
dataset['PROVINCE'].replace('Saskachewan/Alberta','Saskatchewan/Alberta',inplace=True)
dataset['PROVINCE CODE'] = dataset['PROVINCE'].map(lambda x: prov_codes[x])
dataset['# of Rooms'] = dataset['Type of unit'].map(lambda x:Rooms[x])

dataset

/var/folders/_v/4d1j74hd4y3by9wnkl0gym5w0000gn/T/ipykernel_5468/1271277521.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('/Users/ayushchaudhari/Downloads/34100133-eng/34100133.csv')


,YEAR,GEO,DGUID,Type of structure,Type of unit,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,MONTHLY RENT,STATUS,SYMBOL,TERMINATED,DECIMALS,PROVINCE,PROVINCE CODE,# of Rooms
34,1987,"Corner Brook, Newfoundland and Labrador",2011S0504015,Apartment structures of six units and over,Two bedroom units,Dollars,81,units,0,v3824924,2.4.3,480.0,NaN,NaN,NaN,0,Newfoundland and Labrador,709,2
49,1987,"Gander, Newfoundland and Labrador",2011A00051006009,Apartment structures of six units and over,One bedroom units,Dollars,81,units,0,v3824688,3.4.2,370.0,NaN,NaN,NaN,0,Newfoundland and Labrador,709,1
50,1987,"Gander, Newfoundland and Labrador",2011A00051006009,Apartment structures of six units and over,Two bedroom units,Dollars,81,units,0,v3824876,3.4.3,414.0,NaN,NaN,NaN,0,Newfoundland and Labrador,709,2
51,1987,"Gander, Newfoundland and Labrador",2011A00051006009,Apartment structures of six units and over,Three bedroom units,Dollars,81,units,0,v3825064,3.4.4,414.0,NaN,NaN,NaN,0,Newfoundland and Labrador,709,3
69,1987,"Labrador City, Newfoundland and Labrador",2011A00051010032,Apartment structures of six units and over,One bedroom units,Dollars,81,units,0,v3824749,5.4.2,254.0,NaN,NaN,t,0,Newfoundland and Labrador,709,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122355,2022,"Yellowknife, Northwest Territories",2011S0504995,Apartment structures of three units and over,Three bedroom units,Dollars,81,units,0,v3824416,188.1.4,2133.0,NaN,NaN,NaN,0,Northwest Territories,867,3
122356,2022,"Yellowknife, Northwest Territories",2011S0504995,Apartment structures of six units and over,Bachelor units,Dollars,81,units,0,v3824602,188.4.1,1279.0,NaN,NaN,NaN,0,Northwest Territories,867,0
122357,2022,"Yellowknife, Northwest Territories",2011S0504995,Apartment structures of six units and over,One bedroom units,Dollars,81,units,0,v3824790,188.4.2,1563.0,NaN,NaN,NaN,0,Northwest Territories,867,1
122358,2022,"Yellowknife, Northwest Territories",2011S0504995,Apartment structures of six units and over,Two bedroom units,Dollars,81,units,0,v3824978,188.4.3,1820.0,NaN,NaN,NaN,0,Northwest Territories,867,2


In [3]:
grouped = dataset.groupby(['PROVINCE','YEAR','Type of unit'])[['MONTHLY RENT','PROVINCE CODE','# of Rooms']].mean().reset_index()
grouped

,PROVINCE,YEAR,Type of unit,MONTHLY RENT,PROVINCE CODE,# of Rooms
0,Alberta,1987,Bachelor units,318.500000,403.0,0.0
1,Alberta,1987,One bedroom units,372.444444,403.0,1.0
2,Alberta,1987,Three bedroom units,523.625000,403.0,3.0
3,Alberta,1987,Two bedroom units,448.000000,403.0,2.0
4,Alberta,1988,Bachelor units,309.000000,403.0,0.0
...,...,...,...,...,...,...
1954,Saskatchewan/Alberta,2021,Two bedroom units,1000.142857,403.0,2.0
1955,Saskatchewan/Alberta,2022,Bachelor units,703.000000,403.0,0.0
1956,Saskatchewan/Alberta,2022,One bedroom units,956.500000,403.0,1.0
1957,Saskatchewan/Alberta,2022,Three bedroom units,1262.500000,403.0,3.0


In [4]:
province = 'Manitoba' #change this to any province you'd like

province_grouped = grouped.loc[grouped['PROVINCE'] == province]
px.line(province_grouped,x='YEAR',y='MONTHLY RENT',color='Type of unit', title='Average Monthly Rent of ' + province + ' through the Years')

In [5]:
average_per_province = grouped.groupby(['PROVINCE','YEAR'])['MONTHLY RENT'].mean().reset_index()
px.bar(average_per_province,x='YEAR',y='MONTHLY RENT',color='PROVINCE',title='Average Monthly Rent by Year')

In [6]:
px.bar(grouped,x='PROVINCE',y='MONTHLY RENT',color='Type of unit',barmode='group',animation_frame='YEAR',title='Average Monthly Rent Animation')

In [14]:
specific_year = 2001 # you can change this number to look at a specific year

year_info = grouped.loc[grouped['YEAR'] == specific_year]
year_info.sort_values('MONTHLY RENT',ascending=False,inplace=True)
px.bar(year_info,x='PROVINCE',y='MONTHLY RENT',color='Type of unit',barmode='group',title='Average Monthly Rent in ' + str(specific_year) + ' for each Canadian Province')

/var/folders/_v/4d1j74hd4y3by9wnkl0gym5w0000gn/T/ipykernel_5468/3465613684.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
target = grouped['MONTHLY RENT']
features = grouped[['YEAR','PROVINCE CODE','# of Rooms']]

X_train,X_test,Y_train,Y_test = train_test_split(features,target,test_size=0.33,random_state=42)

model = Ridge(alpha=0.5).fit(X_train,Y_train)
y_pred = model.predict(X_test)

X_test['Y_test'] = Y_test
X_test['Y_pred'] = y_pred

province_of_interest = 'Ontario' #change this to any province you're interested in
province_code = prov_codes[province_of_interest]

X_test = X_test[X_test['PROVINCE CODE'] == province_code]

X_test


,YEAR,PROVINCE CODE,# of Rooms,Y_test,Y_pred
1144,1998,249.0,3.0,693.662069,655.190899
1187,2009,249.0,1.0,651.500000,627.872136
1261,1991,249.0,2.0,563.250000,430.667089
1375,2020,249.0,1.0,964.625000,813.432832
1132,1995,249.0,3.0,671.634483,604.583436
...,...,...,...,...,...
1223,2018,249.0,1.0,836.132184,779.694524
1315,2005,249.0,1.0,656.250000,560.395518
1216,2016,249.0,3.0,1087.625000,958.835676
1286,1998,249.0,0.0,446.916667,335.871709


In [9]:
X_test.sort_values(['YEAR','# of Rooms'],inplace=True)
fig = make_subplots(rows=4,cols=1,shared_xaxes=True,subplot_titles=('Bachelor Units','One Bedroom Units','Two Bedroom Units','Three Bedroom Units'))

for i in range(4):
    fig.append_trace(go.Line(x= X_test[X_test['# of Rooms'] == i]['YEAR'],y= X_test[X_test['# of Rooms'] == i]['Y_test'],line=dict(color='royalblue', width= 3)),row=i+1,col=1)
    fig.append_trace(go.Line(x= X_test[X_test['# of Rooms'] == i]['YEAR'],y= X_test[X_test['# of Rooms'] == i]['Y_pred'],line=dict(color='firebrick', width= 4,dash='dash')),row=i+1,col=1)

fig.update_layout(height=1000,width=1000,title_text='Prediction vs Actual Rent values in ' + province_of_interest,showlegend=False)
fig.show()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


